## By:
## Ahmed Mozammil Iqbal (Roll no. 24100085)
## Javeria Siddique (Roll no. 24100246)

In [1]:
!pip install tk

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:

class TicTacToe:
    def __init__(self):
        self.board = [0 for _ in range(9)]
        self.current_player = -1

    def print_board(self):
        for i in range(0, 9, 3):


            print(str(self.board[i]) + "|" + str(self.board[i + 1]) + "|" + str(self.board[i + 2]))
            if i < 6:
                print("-" * 5)

        print()

    def check_win(self, player):
        win_conditions = [(0, 1, 2), (3, 4, 5), (6, 7, 8),
                        (0, 3, 6), (1, 4, 7), (2, 5, 8),
                        (0, 4, 8), (2, 4, 6)]

        for condition in win_conditions:
            if all(self.board[i] == player for i in condition):
                return True
        return False

    def step(self, position):
        if self.board[position] == 0:
            self.board[position] = self.current_player
            if self.check_win(self.current_player):
                return self.board, self.current_player, True
            elif 0 not in self.board:
                return self.board, 0, True
            self.current_player = 1 if self.current_player == -1 else -1
            return self.board, self.current_player, False
        else:
            print("Cell already occupied. Try again.")
            return self.board, self.current_player, False


    def reset(self):
        self.__init__()


In [4]:
# # If you want to use the GUI version that is up to you. While testing your algorithm we will use a code simliar to the above.

# import tkinter as tk
# from tkinter import messagebox

# class TicTacToeGUI:
#     def __init__(self, root):
#         self.root = root
#         self.root.title("Tic Tac Toe")
#         self.game = TicTacToe()
#         self.buttons = []
#         self.create_board()

#     def create_board(self):
#         for i in range(3):
#             for j in range(3):
#                 button = tk.Button(self.root, text=" ", font=('Arial', 20), width=5, height=2,
#                                    command=lambda row=i, col=j: self.on_click(row, col))
#                 button.grid(row=i, column=j)
#                 self.buttons.append(button)

#     def update_board(self):
#         for i in range(9):
#             self.buttons[i]['text'] = 'X' if self.game.board[i] == -1 else 'O' if self.game.board[i] == 1 else ' '

#     def on_click(self, row, col):
#         position = row * 3 + col
#         board, player, terminated = self.game.step(position)
#         self.update_board()

#         if terminated:
#             if player == -1:
#                 messagebox.showinfo("Game Over", "Player -1 wins")
#             elif player == 1:
#                 messagebox.showinfo("Game Over", "Player 1 wins")
#             elif player == 0:
#                 messagebox.showinfo("Game Over", "It's a draw")
#             self.root.quit()

# if __name__ == '__main__':
#     root = tk.Tk()
#     tic_tac_toe_gui = TicTacToeGUI(root)
#     root.mainloop()


You are required to solve Tic Tac Toe using **Value Iteration**. The optimal policy should give a draw no matter what.  
It should work whether you are player -1 or player 1. It doesn't matter what your turn is.

It also follows that your code should win board states where it can and make optimal decisions.

Our final goal is to solve for 3d Tic Tac Toe, 4 X 4 X 4, using reinforcement learning. But, we will start by solving the 2d case and then gradually build up to the 3d case.

There is no starter code available. You are free to choose your implementation. One suggestion is to give +1 reward for a win, 0 for a draw and -1 for a loss.

# Ans starts here

In [5]:
# X = -1; O = 1; empty box = 0
# initialization functions

#Generate all 3 by 3 states
def generate_combinations(arr, index, allStates):
    if index == len(arr):
        allStates.append(arr.copy())
        return

    for value in [0,-1,1]:
        arr[index] = value
        generate_combinations(arr, index + 1, allStates)

def generate_all_states():
    allStates = []
    generate_combinations(np.zeros(9), 0, allStates)
    return allStates

# Removes all impossible states
def remove_impossible_states(allStates):
  reducedStates=[]

  # Removing all states where numX and numO are inconsistent (We want numX>=numO and numX can be at max 1 greater than numO considering X takes the first game turn)
  for state in allStates:
    numX = np.count_nonzero(state == -1)
    numO = np.count_nonzero(state == 1)

    if numX==numO or numO==(numX-1) or (numX ==0 and numO==0):
      reducedStates.append(state)

  # Removing all terminal states which are seemingly impossible: states where both X and O win
  finalStates=[]
  for state in reducedStates:
    if not (player_wins(state, -1) and player_wins(state,1)):
      finalStates.append(state)
  return finalStates

# player is either -1 (X) or 1 (O)
# we check if X wins or O wins with this function, will be used to get the final derived states
# (We split it in multiple If statement for better readibility)
def player_wins(state, player):
  # Check rows
  if state[0]==player and state[1]== player and state[2]==player:
    return True
  elif state[3]==player and state[4]== player and state[5]==player:
    return True
  elif state[6]==player and state[7]== player and state[8]==player:
    return True

  # check columns
  elif state[0]==player and state[3]== player and state[6]==player:
    return True
  elif state[1]==player and state[4]== player and state[7]==player:
    return True
  elif state[2]==player and state[5]== player and state[8]==player:
    return True

  # check diagonals
  elif state[0]==player and state[4]== player and state[8]==player:
    return True
  elif state[2]==player and state[4]== player and state[6]==player:
    return True
  else:
    return False


# Initialises the value table to an array of size number of States
def initial_value_table(numberOfStates):
  return np.zeros(numberOfStates)

#initialises the policy table to an array of size number of states (policy table will contain the next action, for even indexes:0,2,4... it is for X else it is for Y)
def initial_policy_table(numberOfStates):
  return np.full(numberOfStates, -1)

#initialise the Qtable to array of null
def initial_qtable(rows, columns):
  return np.full((rows, columns), np.nan)

In [6]:
# checksa particular state to see whose turn it is now
def check_turn(state):
  if np.all(state == 0):
    return -1
  elif np.count_nonzero(state == -1)>np.count_nonzero(state == 1):
    return 1
  else:
    return -1

# finds whose turn it is for all states
def whose_turn(all_states):
  turns_array=[]
  for state in all_states:
    turns_array.append(check_turn(state))

  return turns_array

#computes the optimal policy from the final qtable given input of all states and turns_array (found in whose_turn)
def optimal_policy_calculate(qtable, all_states, turns_array):
  policies={}

  for i in range(len(all_states)):
    if turns_array[i]==-1:
      row=qtable[i]
      if np.all(np.isnan(row)) or player_wins(all_states[i], 1) or player_wins(all_states[i], -1):
        policies[tuple(all_states[i])]=None
      else:
        policies[tuple(all_states[i])]=np.nanargmax(row)

    elif turns_array[i]==1:
      row=qtable[i]
      if np.all(np.isnan(row)) or player_wins(all_states[i], -1) or player_wins(all_states[i], 1):
        policies[tuple(all_states[i])]=None
      else:
        policies[tuple(all_states[i])]=np.nanargmin(row)

  return policies

In [7]:
# returns index of the state in the set of all final possible states
def find_index_in_set_of_states(state, all_possible_states):
  all_possible_states_np = np.array(all_possible_states)
  # Use NumPy's where function to find the indices where the condition is true
  indices = np.where(np.all(all_possible_states_np == state, axis=1))[0]
  return indices[0]

# returns next possible states as well as the locations where the player put their value

def next_possible_states(state, playeraction):
    zero_indexes = np.where(np.array(state) == 0)[0]
    nextstates = [np.copy(state) for _ in zero_indexes]

    for i, pos_nextaction in enumerate(zero_indexes):
        nextstates[i][pos_nextaction] = playeraction

    return nextstates, zero_indexes


# Calculates the reward for the given state
# X is notated by -1 and O is notated by 1
def reward_function(nextstate, player, current_state):
  if player_wins(nextstate, player):  #if player wins, give reward
    if player == -1:
      return 1
    elif player == 1:
      return -1
  else:
    return 0


# find all states in set of states which are terminating, on terminating states, there are no next states
def find_termination_states(all_possible_states):
  termination_states=[]

  for state in all_possible_states:
    # All winning states
    if player_wins(state, -1) or player_wins(state, 1):
      termination_states.append(state)

    # All draw states (basically if there are no 0s means no empty space hence draw)
    elif np.count_nonzero(state == 0) == 0:
      termination_states.append(state)

  return termination_states

# for array of all next states passed in the function, the bellman equation value of all of the next states is returned
def calculate_bellman_values(all_states, currentstate, nextstates, player, action_locations, value_table, q_table, gamma, state_to_index):
  # index_cur_state=find_index_in_set_of_states(currentstate, all_states)
  index_cur_state=state_to_index[tuple(currentstate)]
  next_state_bellman_values = [reward_function(state, player, currentstate) + gamma * value_table[state_to_index[tuple(state)]] for state in nextstates]

  i=0
  for col in action_locations:
    q_table[index_cur_state][col]=next_state_bellman_values[i]
    i+=1

  return value_table, q_table

import copy

#This function updates all states according to the current value table and q_table for only the current iteration using bellman's equation
def update_all_states(all_states, curr_player, value_table, q_table, gamma, termination_states, filtered_states, all_turns, state_to_index):
  initial= all_states[0]

  for i, state in enumerate(filtered_states):
    # state =all_states[i]
    # if not any(np.array_equal(state, term_state) for term_state in termination_states):
    playeraction= all_turns[tuple(state)]
    nextstates, action_locations=next_possible_states(state, playeraction)
    value_table, q_table= calculate_bellman_values(all_states, state, nextstates, playeraction, action_locations, value_table, q_table, gamma, state_to_index)


  for i, state in enumerate(filtered_states):
    player= all_turns[tuple(state)]
    index_cur_state=state_to_index[tuple(state)]

    if player==-1:
      value_table[index_cur_state]=np.nanmax(q_table[index_cur_state])

    # if player is 1 (O), then we find min and store in the respective index of value table
    elif player==1:
      value_table[index_cur_state]=np.nanmin(q_table[index_cur_state])

  return value_table, q_table




# This function runs the value iteration algorithm continuously over all states till the value table converges to a specific threshold i.e., 10^(-1)
def value_iteration(gamma):
  initial_player = -1
  all_states = remove_impossible_states(generate_all_states())
  prev_value_table = initial_value_table(len(all_states))
  value_table = initial_value_table(len(all_states))
  q_table = initial_qtable(len(all_states), 9)
  i = 0
  termination_states = find_termination_states(all_states)
  filtered_states = [state for state in all_states if not any(np.array_equal(state, term_state) for term_state in termination_states)]

  all_turns={}
  for state in filtered_states:
    all_turns[tuple(state)]=check_turn(state)

  state_to_index={}
  i=0
  for state in all_states:
    state_to_index[tuple(state)]=i
    i+=1

  while True:
    prev_value_table = copy.deepcopy(value_table)

    #calculates q table and value table for all states for current iteration
    value_table, q_table = update_all_states(all_states, initial_player, value_table, q_table, gamma, termination_states, filtered_states, all_turns, state_to_index)

    print(np.max(value_table - prev_value_table))
    if i == 0:
      i+=1
      continue
    else:
      i+=1
      if np.max(value_table - prev_value_table) <= 10**-18:  #convergence achieved
        value_table_list = value_table.tolist()
        q_table_list = q_table.tolist()

        save_table_to_file(value_table_list, "value_table_Deliverable1.txt")
        save_table_to_file(q_table_list, "q_table_Deliverable1.json")
        return value_table, q_table

import json

def save_table_to_file(table, filename):
    with open(filename, 'w') as file:
        json.dump(table, file)

#calculates optimal policy from the final value table
def find_optimal_tic_tac_toe(gamma, states):
  value_table, qtable=value_iteration(gamma)
  print(np.array(value_table))
  turns_table=whose_turn(states)
  return optimal_policy_calculate(qtable,states, turns_table)

In [8]:
all_states = remove_impossible_states(generate_all_states())  #all possible states of the game
optimal=find_optimal_tic_tac_toe(0.99,all_states)  #calculates optimal action for each state i.e., optimal policy
print(optimal)

1.0
0.99
0.9801
0.9702989999999999
0.96059601
0.0
[0. 0. 0. ... 0. 0. 0.]
{(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0): 0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0): 4, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0): 1, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 1.0): 0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, -1.0): 2, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0): 4, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 1.0): 0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, -1.0, 1.0): 2, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 1.0, 0.0): 0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 1.0, -1.0): 4, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, -1.0): 0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, -1.0, 0.0): 0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, -1.0, -1.0): 0, (0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0): 2, (0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 1.0): 0, (0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, -1.0, 1.0): 1, (0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 1.0, 0.0): 4, (0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 1.0, -1.0): 2, (0.

In [10]:
#Applying tte policy
i=0
env = TicTacToe()
while True:
    env.print_board() # You can comment this part out if you don't want to see the board
    if i==0:
      i+=1
      position = int(input(f"Player {env.current_player}, enter your move (0-8): "))  #initializing the first action on our own to test the system
    else:
      position=optimal[tuple(env.board)]  #optimal policy decides the next step on its own

    if not (0 <= position < 9):
        print("Invalid position. Please try again.")
        continue

    board, player, terminated = env.step(position)

    if terminated:
        if player == -1:
            env.print_board()
            print("Player -1 wins")
        elif player == 1:
            env.print_board()
            print("Player 1 wins")
        elif player == 0:
            print("It's a draw")
        break

0|0|0
-----
0|0|0
-----
0|0|0

0|0|0
-----
0|0|0
-----
-1|0|0

0|0|0
-----
0|1|0
-----
-1|0|0

-1|0|0
-----
0|1|0
-----
-1|0|0

-1|0|0
-----
1|1|0
-----
-1|0|0

-1|0|0
-----
1|1|-1
-----
-1|0|0

-1|1|0
-----
1|1|-1
-----
-1|0|0

-1|1|0
-----
1|1|-1
-----
-1|-1|0

-1|1|0
-----
1|1|-1
-----
-1|-1|1

It's a draw
